<h1> Objective of the challenge: </h1>
<p> we aim to build a model that predict the estimated duration of the travel. <p>

# 1. Data cleaning and preparation
This dataset coming from mobility startup that lets any user to book a ride to from any point A to any point B  within the city using a smartphone. Ride value is calculated at the time of request automatically  by the app, considering distance, estimated travel time, and current car availability (demand /  offer balance).

In [1]:
 #import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import missingno as msno
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import datetime

%matplotlib inline

In [2]:
#loading_data
data=pd.read_csv("uber_peru_2010.csv",sep=";")
data.head()

,journey_id,user_id,driver_id,taxi_id,icon,start_type,start_at,start_lat,start_lon,end_at,...,price_distance,price_duration,distance,duration,cost,cost_distance,cost_duration,source,driver_score,rider_score
0,23a1406fc6a11d866e3c82f22eed4d4c,0e9af5bbf1edfe591b54ecdfd7e91e26,583949a89a9ee17d19e3ca4f137b6b4c,b12f4f09c783e29fe0d0ea624530db56,executive,asap,16/11/2010 16:44,"-12,13983536","-77,02355957",16/11/2010 17:29,...,3626.0,195.0,11331.0,234.0,0.0,0.0,0.0,iPhone,5.0,5.0
1,dd2af4715d0dc16eded53afc0e243577,a553c46e3a22fb9c326aeb3d72b3334e,NaN,NaN,executive,asap,01/06/2010 0:34,"-12,13874817","-76,99536133",01/06/2010 0:37,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,iPhone,NaN,NaN
2,dd91e131888064bf7df3ce08f3d4b4ad,a553c46e3a22fb9c326aeb3d72b3334e,NaN,NaN,executive,asap,31/05/2010 5:01,"-12,12453079","-77,02780151",31/05/2010 5:04,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,iPhone,NaN,NaN
3,dd2af4715d0dc16eded53afc0e2466d0,a553c46e3a22fb9c326aeb3d72b3334e,NaN,NaN,executive,asap,01/06/2010 0:29,"-12,13885117","-76,99530029",01/06/2010 0:32,...,NaN,NaN,0.0,0.0,0.0,NaN,NaN,iPhone,NaN,NaN
4,85b7eabcf5d84e42dc7629b7d27781af,56772d544fdfa589a020a1ff894a86f7,d665fb9f75ef5d9cd0fd89479380ba78,0accdd3aa5a322f4129fa20b53278c69,executive,reserved,11/09/2010 23:55,"-12,08995438","-76,92626953",12/09/2010 1:07,...,7665.0,562.0,30270.0,715.0,6173.0,5756.0,417.0,iPhone,4.0,5.0


In [ ]:
#number of rows and columns
data.shape

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23111 entries, 0 to 23110
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   journey_id        23111 non-null  object 
 1   user_id           23111 non-null  object 
 2   driver_id         19726 non-null  object 
 3   taxi_id           19726 non-null  object 
 4   icon              23111 non-null  object 
 5   start_type        23111 non-null  object 
 6   start_at          23111 non-null  object 
 7   start_lat         23111 non-null  object 
 8   start_lon         23111 non-null  object 
 9   end_at            22835 non-null  object 
 10  end_lat           23111 non-null  object 
 11  end_lon           23111 non-null  object 
 12  end_state         23099 non-null  object 
 13  driver_start_lat  19621 non-null  object 
 14  driver_start_lon  19621 non-null  object 
 15  arrived_at        17716 non-null  object 
 16  currency          23111 non-null  object

In [3]:
data.describe()

,price,price_distance,price_duration,distance,duration,cost,cost_distance,cost_duration,driver_score,rider_score
count,22713.000000,1.994100e+04,19941.000000,2.284800e+04,22848.000000,21760.000000,1.803800e+04,18038.000000,7650.000000,15390.000000
mean,2752.738784,3.404748e+03,606.005617,1.088395e+04,638.831145,2655.032445,2.099408e+03,401.550504,3.933333,4.755491
std,3025.390309,4.851010e+04,1553.548938,2.025735e+05,1788.661444,17124.801935,3.300314e+04,1148.614398,1.744948,0.840558
min,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1700.000000,8.270000e+02,30.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,4.000000,5.000000
50%,1911.000000,1.754000e+03,247.000000,4.660000e+03,218.000000,1450.000000,1.048000e+03,127.000000,5.000000,5.000000
75%,3597.000000,3.256000e+03,628.000000,9.290250e+03,667.000000,2381.250000,2.230000e+03,414.000000,5.000000,5.000000
max,55974.000000,4.491910e+06,69839.000000,1.403722e+07,83807.000000,525000.000000,3.368792e+06,48887.000000,5.000000,5.000000


## Feature selection and Trait missing values
Feature Selection is one of the core concepts in machine learning which hugely impacts the performance of your model. The data features that you use to train your machine learning models have a huge influence on the performance you can achieve.  
<b> Irrelevant or partially relevant features can negatively impact model performance. </b>

Missing values are one of the most common problems you can encounter when you try to prepare your data for machine learning. The reason for the missing values might be human errors, interruptions in the data flow, privacy concerns, and so on. Whatever is the reason, missing values affect the performance of the machine learning models.

In [ ]:
l=[]
for col in data.columns :
    if (data[col].isnull().sum()!=0):
        l.append([col,data[col].isnull().sum(),"{:.0%}".format(data[col].isnull().sum()/23111)])
l.sort(key= lambda x:x[1],reverse=True)
l

In [ ]:
msno.matrix(data)

In [ ]:
msno.bar(data)

In [5]:
chosen_features =  ['icon','start_type','start_at', 'start_lat', 'start_lon', 'end_at', 'end_lat', 'end_lon',
                    'end_state','source','distance' ,'cost_duration','price_duration','duration']

In [6]:
features=data[chosen_features]

We take a look at the distribution of the price_duration and cost_duration column to see if it's skewed or symmetrical. This will help us determine what value to replace the NaN values.

In [ ]:
copy = features.copy()
copy.dropna(inplace = True)
sns.distplot(copy["price_duration"])

In [ ]:
features["price_duration"].fillna(features["price_duration"].mean())
features["cost_duration"].fillna(features["cost_duration"].mean())

In [7]:
features=features.dropna()

In [8]:
features.drop(features[features['distance']==0].index , inplace=True)

In [9]:
features.drop(features[features['duration']==0].index , inplace=True)

## Feature engineering
Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work.

In [ ]:
# Create 4 new variables, dayofweek number and name, month of day and hour of day.
features['start_at'] = pd.to_datetime(features['start_at'], format="%d/%m/%Y %H:%M")
features['end_at'] = pd.to_datetime(features['end_at'], format="%d/%m/%Y %H:%M")
features['Month'] = features['start_at'].dt.month
features['Hour'] = features['start_at'].dt.hour
features['Day'] = features['start_at'].dt.day


In [ ]:
features['time']=(features['end_at'] - features['start_at']).astype('timedelta64[m]')

In [ ]:
features['DayOfWeekNum'] = features['start_at'].dt.dayofweek
week=["lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"]
def transfer(i):
    return week[i]
    
features['DayOfWeek']=features['DayOfWeekNum'].apply(transfer)

In [ ]:
features.drop(["start_at","end_at"], axis = 1, inplace = True)
features.head()

In [ ]:
def convert_lat(lat):
    lat=lat.replace(',','.')
    return float(lat)

In [ ]:
features['end_lat']=features['end_lat'].apply(convert_lat)
features['end_lon']=features['end_lon'].apply(convert_lat)
features['start_lat']=features['start_lat'].apply(convert_lat)
features['start_lon']=features['start_lon'].apply(convert_lat) 

In [ ]:
features.head(10)

# 2. Exploratory data analysis
Data visualization is the graphic representation of data. It involves producing images that communicate relationships among the represented data to viewers of the images.

In [ ]:
cols=['start_type','icon']
for feature in cols:
    features[feature].hist(bins=25)
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.title(feature)
    plt.show()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(features.corr(), annot = True,fmt='.1g', vmin=-1, vmax=1, center= 0, cmap= 'coolwarm')

In [ ]:
fig, (ax1, ax2 , ax3) = plt.subplots(nrows=3, ncols=1, figsize=(20,20))


sns.scatterplot(x="duration",y="time",data=features,ax=ax1)
sns.scatterplot(x="cost_duration",y="time",data=features,ax=ax2)
sns.scatterplot(x="price_duration",y="time",data=features,ax=ax3)

In [ ]:
#handling outliers
features.drop(features[ (features['time']>200) & (features['duration']<400) ].index , inplace=True)

In [ ]:
fig, (ax1, ax2 , ax3) = plt.subplots(nrows=3, ncols=1, figsize=(20,20))


sns.scatterplot(x="duration",y="time",data=features,ax=ax1)
sns.scatterplot(x="cost_duration",y="time",data=features,ax=ax2)
sns.scatterplot(x="price_duration",y="time",data=features,ax=ax3)

In [ ]:
# Lets we visualize total riders for each day of week
features_weekdays = features.pivot_table(index=['DayOfWeekNum','DayOfWeek'],
                                  values='time',
                                  aggfunc='count')
features_weekdays.plot(kind='bar', figsize=(15,6))
plt.ylabel('Total riders')
#plt.set_xticklabels(x_labels)
plt.title('Riders by Week Day');

In [ ]:
uber_monthdays = features.pivot_table(index=['Month'],
                                  values='time',
                                  aggfunc='count')
uber_monthdays.plot(kind='bar', figsize=(15,6))
plt.ylabel('Total Riders')
plt.title('Riders by Month Day');

In [ ]:
#group the data by Weekday and hour
summary = features.groupby(['DayOfWeek', 'Hour'])['time'].count()
#convert to dataframe
summary = pd.DataFrame(summary)
#reset index
summary = summary.reset_index()
#rename last column
summary=summary.rename(columns = {'time':'Counts'})
#browse data
summary.head()

In [ ]:
tableau_color_blind = [(0, 107, 164), (255, 128, 14), (171, 171, 171), (89, 89, 89),
             (95, 158, 209), (200, 82, 0), (137, 137, 137), (163, 200, 236),
             (255, 188, 121), (207, 207, 207)]

for i in range(len(tableau_color_blind)):  
    r, g, b = tableau_color_blind[i]  
    tableau_color_blind[i] = (r / 255., g / 255., b / 255.)

In [ ]:
sns.set_style('whitegrid')

## set palette   
current_palette = sns.color_palette(tableau_color_blind)
plt.figure(figsize=(15,6))
ax = sns.pointplot(x="Hour", y="Counts", hue="DayOfWeek", data=summary, palette = current_palette,)
handles,labels = ax.get_legend_handles_labels()
#reordering legend content
handles = [handles[1], handles[5], handles[6], handles[4], handles[0], handles[2], handles[3]]
labels = [labels[1], labels[5], labels[6], labels[4], labels[0], labels[2], labels[3]]
ax.yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
ax.set_xlabel('Hour', fontsize = 12)
ax.set_ylabel('Count of Uber Pickups', fontsize = 12)
ax.set_title('Hourly Uber Pickups By Day of the Week in Peru (Jan-X 2010)', fontsize=16)
ax.tick_params(labelsize = 8)
ax.legend(handles,labels,loc=0, title="Legend", prop={'size':8})
ax.get_legend().get_title().set_fontsize('8')
plt.show()

It seems that peak hour is 1pm, very far as normal peak hour between 7am and 10am, and we see other peak after 9pm.

It's very strange if time variable is relative to Lima (Perú), maybe it very similar with difference timezone, like as Europe timezone, so it make sense. The time difference is 6 or 7 hours Spain and Peru, ( winter or summer time). So I want to check it.

In [ ]:
g = sns.lmplot(x="duration", y="time", col="start_type", data=features)
g = sns.lmplot(x="cost_duration", y="time", col="start_type", data=features)
g = sns.lmplot(x="price_duration", y="time", col="start_type", data=features)

# 3.Data preprocessing

## Encode data
One-hot encoding is one of the most common encoding methods in machine learning. This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

In [ ]:
#Encode data
def oneHotEncode(df, col):
    dfDummies = pd.get_dummies(df[col], prefix = col)
    df = pd.concat([df, dfDummies], axis=1)
    
    # write youe code here
    df= df.drop([col], axis=1)
    return df

In [ ]:
features.head()

In [ ]:
# applying the function to our data
features = oneHotEncode(features, 'start_type')
#features = oneHotEncode(features, 'DayOfWeekNum')
features.head()

## Scaling
In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. In real life, it is nonsense to expect age and income columns to have the same range. But from the machine learning point of view, how these two columns can be compared?  
<b> Scaling </b> solves this problem. The continuous features become identical in terms of the range, after a scaling process.

In [ ]:
#scale data
def scale(df, cols):     
    for col in cols:
        
        # write your code here: 
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
    return df
features = scale(features, ["cost_duration","price_duration","distance","duration","start_lat","start_lon","end_lat","end_lon"]) 
features.head()

# 4. Model fitting and predicting using SVR and XGboost

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn import svm
from sklearn import linear_model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
#split data
labels=features["time"]
features.drop(["time","icon","end_state","source","Month","DayOfWeek","DayOfWeekNum","Day","Hour"], axis=1, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)

In [ ]:
features.drop(["DayOfWeek"], axis=1, inplace = True)

### LinearRegression

In [ ]:
clf = LinearRegression()
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score_lr = r2_score(y_test, pred)
rmse_lr = sqrt(mean_squared_error(y_test, pred))
print('Coefficient of determination: %.2f' % score_lr)
print('Mean squared error: %.2f' % rmse_lr)

### GaussianNB

In [ ]:
clf = GaussianNB()
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score_gnb = r2_score(y_test, pred)
rmse_gnb = sqrt(mean_squared_error(y_test, pred))
print('Coefficient of determination: %.2f' % score_gnb)
print('Mean squared error: %.2f' % rmse_gnb)

### RandomForestRegressor

In [ ]:
clf  = RandomForestRegressor(max_depth=10, random_state=42, n_estimators=100, min_samples_split = 150)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score_rfr = r2_score(y_test, pred)
rmse_rfr = sqrt(mean_squared_error(y_test, pred))
print('Coefficient of determination: %.2f' % score_rfr)
print('Mean squared error: %.2f' % rmse_rfr)

### SGDRegressor

In [ ]:
clf = linear_model.SGDRegressor()
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score_sgd = r2_score(y_test, pred)
rmse_sgd = sqrt(mean_squared_error(y_test, pred))
print('Coefficient of determination: %.2f' % score_sgd)
print('Mean squared error: %.2f' % rmse_sgd)

### 1) SVR Regressor

In [ ]:
from sklearn.svm import SVR

# First, let's choose which kernel is the best for our data

for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(X_train, y_train)
    confidence = clf.score(X_train, y_train)
    print(k,confidence)

In [ ]:
from sklearn.svm import SVR

clf = svm.SVR(kernel ='poly')
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score_svr = r2_score(y_test, pred)
rmse_svr = sqrt(mean_squared_error(y_test, pred))
print('Coefficient of determination: %.2f' % score_svr)
print('Mean squared error: %.2f' % rmse_svr)

In [ ]:
# plot prediction and actual data
plt.plot(y_test, pred, '.')

 ### 2) XGBoost regressor 

XGBoost is well known to provide better solutions than other machine learning algorithms. In fact, since its inception, it has become the "state-of-the-art” machine learning algorithm to deal with structured data.

In [ ]:
import xgboost as xgb

Now you will convert the dataset into an optimized data structure called Dmatrix that XGBoost supports and gives it acclaimed performance and efficiency gains. You will use this later in the tutorial.

### Tuning Parameters with GridSearchCV

We can improve the accuracy of our model by turning the hyperparameters of our Random Forest model. We will run a GridSearchCV to find the best parameters for the model and use that model to train and test our data.

### XGBoost's hyperparameters
At this point, before building the model, you should be aware of the tuning parameters that XGBoost provides. Well, there are a plethora of tuning parameters for tree-based learners in XGBoost and you can read all about them here. But the most common ones that you should know are:

<b> learning_rate </b>: step size shrinkage used to prevent overfitting. Range is [0,1] 

<b> max_depth </b>: determines how deeply each tree is allowed to grow during any boosting round.

<b> subsample </b>: percentage of samples used per tree. Low value can lead to underfitting.

<b> colsample_bytree </b>: percentage of features used per tree. High value can lead to overfitting.

<b> n_estimators </b>: number of trees you want to build.
    
objective: determines the loss function to be used like reg:linear for regression problems, reg:logistic for classification problems with only decision, binary:logistic for classification problems with probability.
XGBoost also supports regularization parameters to penalize models as they become more complex and reduce them to simple (parsimonious) models.

<b> gamma </b>: controls whether a given node will split based on the expected reduction in loss after the split. A higher value leads to fewer splits. Supported only for tree-based learners.

<b> alpha </b>: L1 regularization on leaf weights. A large value leads to more regularization.

<b> lambda </b>: L2 regularization on leaf weights and is smoother than L1 regularization.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Various hyper-parameters to tune
xgb1 = xgb.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Great, now that we have the optimal parameters for our XGboost regressor, we can build a new model with those parameters to fit and use on the test set.

In [ ]:
rf_clf = xgb_grid.best_estimator_

rf_clf.fit(X_train, y_train)

In [ ]:
#Compute the rmse by invoking the mean_sqaured_error function from sklearn's metrics module.

pred = rf_clf.predict(X_test)
rmse_xgb = sqrt(mean_squared_error(y_test, pred))

print(rmse_xgb)

In [ ]:
score_xgb=r2_score(y_test, pred)

print('Coefficient of determination: %.2f'
      % score_xgb)

In [ ]:
# plot prediction and actual data
plt.plot(y_test, pred, '.')

## Evaluating Model Performances

In [ ]:
model_performance = pd.DataFrame({
    "Model": ["LinearRegression", "SVR", "GaussianNB", 
              "RandomForestRegressor", "SGDRegressor", "XGBoostRegressor"],
    "Coefficient of determination": [score_lr, score_svr,score_gnb, score_rfr, 
              score_sgd, score_xgb],
    "Root-mean-square error": [rmse_lr, rmse_svr,rmse_gnb, rmse_rfr, 
              rmse_sgd, rmse_xgb]
})

model_performance.sort_values(by="Coefficient of determination", ascending=False)